In [1]:
from coffea import hist
from coffea.analysis_objects import JaggedCandidateArray
import coffea.processor as processor
import uproot

import numpy as np
np.seterr(divide='ignore', invalid='ignore', over='ignore')
import matplotlib.pyplot as plt
from FireHydrant.Tools.metfilter import MetFilters
from FireHydrant.Tools.correction import get_pu_weights_function, get_ttbar_weight, get_nlo_weight_function

# import pandas as pd
import awkward
from uproot_methods import TLorentzVectorArray
from FireHydrant.Tools.trigger import Triggers

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import json
import os
from os.path import join


########## data ##########
datasets=json.load(open(join(os.getenv('FH_BASE'), 'Notebooks/Data/Samples/control_data2018_v2.json')))
####################################

## 4mu channel

In [7]:
ptbinning = np.concatenate([np.arange(0, 300, 10),])
massbinning = np.concatenate([np.arange(0, 20, 1),])

class LeptonJetProcessor(processor.ProcessorABC):
    def __init__(self):
        dataset_axis = hist.Cat('dataset', 'backgrounds')
        
        self._accumulator = processor.dict_accumulator({
            'run': processor.column_accumulator(np.zeros(shape=(0,))),
            'lumi': processor.column_accumulator(np.zeros(shape=(0,))),
            'event': processor.column_accumulator(np.zeros(shape=(0,))),
        })
        self.pucorrs = get_pu_weights_function()
        self.nlo_w = get_nlo_weight_function('w')
        self.nlo_z = get_nlo_weight_function('z')
    
    @property
    def accumulator(self):
        return self._accumulator
    
    def process(self, df):
        output = self.accumulator.identity()
        if df.size==0: return output
        
        dataset = df['dataset']
        run = df['run']
        lumi = df['lumi']
        event = df['event']
        ## construct weights ##
        wgts = processor.Weights(df.size)
        
        triggermask = np.logical_and.reduce([df[t] for t in Triggers])
        wgts.add('trigger', triggermask)
        cosmicpairmask = df['cosmicveto_result']
        wgts.add('cosmicveto', cosmicpairmask)
        
        # ...bla bla, other weights goes here
        
        weight = wgts.weight()
        ########################
        
        leptonjets = JaggedCandidateArray.candidatesfromcounts(
            df['pfjet_p4'],
            px=df['pfjet_p4.fCoordinates.fX'],
            py=df['pfjet_p4.fCoordinates.fY'],
            pz=df['pfjet_p4.fCoordinates.fZ'],
            energy=df['pfjet_p4.fCoordinates.fT'],
        )
        ljdautype = awkward.fromiter(df['pfjet_pfcand_type'])
        npfmu = (ljdautype==3).sum()
        ndsa = (ljdautype==8).sum()
        isegammajet = (npfmu==0)&(ndsa==0)
        ispfmujet = (npfmu>=2)&(ndsa==0)
        isdsajet = ndsa>0
        label = isegammajet.astype(int)*1+ispfmujet.astype(int)*2+isdsajet.astype(int)*3
        leptonjets.add_attributes(label=label)
        nmu = ((ljdautype==3)|(ljdautype==8)).sum()
        leptonjets.add_attributes(ismutype=(nmu>=2), iseltype=(nmu==0))
        ljdaucharge = awkward.fromiter(df['pfjet_pfcand_charge']).sum()
        leptonjets.add_attributes(qsum=ljdaucharge)
        leptonjets.add_attributes(isneutral=(leptonjets.iseltype | (leptonjets.ismutype&(leptonjets.qsum==0))))
        leptonjets = leptonjets[leptonjets.isneutral]
                
        twoleptonjets = leptonjets.counts>=2
        dileptonjets = leptonjets[twoleptonjets]
        wgt = weight[twoleptonjets]
        
        run = run[twoleptonjets]
        lumi = lumi[twoleptonjets]
        event = event[twoleptonjets]
        
        if dileptonjets.size==0: return output
        lj0 = dileptonjets[dileptonjets.pt.argmax()]
        lj1 = dileptonjets[dileptonjets.pt.argsort()[:, 1:2]]
        
        ## channel def ##
        singleMuljEvents = dileptonjets.ismutype.sum()==1
        muljInLeading2Events = (lj0.ismutype | lj1.ismutype).flatten()
        channel_2mu2e = (singleMuljEvents&muljInLeading2Events).astype(int)*1
        
        doubleMuljEvents = dileptonjets.ismutype.sum()==2
        muljIsLeading2Events = (lj0.ismutype & lj1.ismutype).flatten()
        channel_4mu = (doubleMuljEvents&muljIsLeading2Events).astype(int)*2
        
        channel_ = channel_2mu2e + channel_4mu
        ###########
        
        output['run'] += processor.column_accumulator(run[(channel_==2)&wgt.astype(bool)])
        output['lumi'] += processor.column_accumulator(lumi[(channel_==2)&wgt.astype(bool)])
        output['event'] += processor.column_accumulator(event[(channel_==2)&wgt.astype(bool)])
        
        
        return output
    
    def postprocess(self, accumulator):
        return accumulator

In [9]:
outputs={}
for k in datasets:
    outputs[k] = processor.run_uproot_job({k: datasets[k]},
                                  treename='ffNtuplizer/ffNtuple',
                                  processor_instance=LeptonJetProcessor(),
                                  executor=processor.futures_executor,
                                  executor_args=dict(workers=12, flatten=True),
                                  chunksize=500000,
                                 )

Processing: 100%|██████████| 1255/1255 [00:39<00:00, 31.98items/s]


In [10]:
for k, output in outputs.items():
    runs, lumis, events = output['run'].value.astype(int), output['lumi'].value.astype(int), output['event'].value.astype(int)
    print(f'\n{k}')
    if runs.size==0: continue
    for r,l,e in np.nditer([runs, lumis, events]):
        print('{}:{}:{}'.format(r, l, e))


A

B

C
319639:1086:1545785609
319849:286:476992516
319908:30:42436699

D
321218:862:1277227109
320804:281:397630839
320804:299:429320095
321305:1947:3032841981
321457:635:1012997600
321457:1439:2262419942
322204:135:253477024
322332:105:133041368
322431:103:167082712
322625:911:1606326814
322625:806:1431138353
323727:591:1068507693
323725:85:128975908
323841:325:539760911
323778:464:894412541
324021:204:306627612
324077:188:272859904
324237:143:212400533
324791:112:201957009
324835:304:529169561
324878:120:133811675
324878:1049:1909816543


## 2mu2e channel

In [3]:
ptbinning = np.concatenate([np.arange(0, 300, 10),])
massbinning = np.concatenate([np.arange(0, 20, 1),])

class LeptonJetProcessor(processor.ProcessorABC):
    def __init__(self):
        dataset_axis = hist.Cat('dataset', 'backgrounds')
        
        self._accumulator = processor.dict_accumulator({
            'run': processor.column_accumulator(np.zeros(shape=(0,))),
            'lumi': processor.column_accumulator(np.zeros(shape=(0,))),
            'event': processor.column_accumulator(np.zeros(shape=(0,))),
        })
        self.pucorrs = get_pu_weights_function()
        self.nlo_w = get_nlo_weight_function('w')
        self.nlo_z = get_nlo_weight_function('z')
    
    @property
    def accumulator(self):
        return self._accumulator
    
    def process(self, df):
        output = self.accumulator.identity()
        if df.size==0: return output
        
        dataset = df['dataset']
        run = df['run']
        lumi = df['lumi']
        event = df['event']
        ## construct weights ##
        wgts = processor.Weights(df.size)
        
        triggermask = np.logical_and.reduce([df[t] for t in Triggers])
        wgts.add('trigger', triggermask)
        cosmicpairmask = df['cosmicveto_result']
        wgts.add('cosmicveto', cosmicpairmask)
        
        # ...bla bla, other weights goes here
        
        weight = wgts.weight()
        ########################
        
        leptonjets = JaggedCandidateArray.candidatesfromcounts(
            df['pfjet_p4'],
            px=df['pfjet_p4.fCoordinates.fX'],
            py=df['pfjet_p4.fCoordinates.fY'],
            pz=df['pfjet_p4.fCoordinates.fZ'],
            energy=df['pfjet_p4.fCoordinates.fT'],
        )
        ljdautype = awkward.fromiter(df['pfjet_pfcand_type'])
        npfmu = (ljdautype==3).sum()
        ndsa = (ljdautype==8).sum()
        isegammajet = (npfmu==0)&(ndsa==0)
        ispfmujet = (npfmu>=2)&(ndsa==0)
        isdsajet = ndsa>0
        label = isegammajet.astype(int)*1+ispfmujet.astype(int)*2+isdsajet.astype(int)*3
        leptonjets.add_attributes(label=label)
        nmu = ((ljdautype==3)|(ljdautype==8)).sum()
        leptonjets.add_attributes(ismutype=(nmu>=2), iseltype=(nmu==0))
        ljdaucharge = awkward.fromiter(df['pfjet_pfcand_charge']).sum()
        leptonjets.add_attributes(qsum=ljdaucharge)
        leptonjets.add_attributes(isneutral=(leptonjets.iseltype | (leptonjets.ismutype&(leptonjets.qsum==0))))
        leptonjets = leptonjets[leptonjets.isneutral]
                
        twoleptonjets = leptonjets.counts>=2
        dileptonjets = leptonjets[twoleptonjets]
        wgt = weight[twoleptonjets]
        
        run = run[twoleptonjets]
        lumi = lumi[twoleptonjets]
        event = event[twoleptonjets]
        
        if dileptonjets.size==0: return output
        lj0 = dileptonjets[dileptonjets.pt.argmax()]
        lj1 = dileptonjets[dileptonjets.pt.argsort()[:, 1:2]]
        
        ## channel def ##
        singleMuljEvents = dileptonjets.ismutype.sum()==1
        muljInLeading2Events = (lj0.ismutype | lj1.ismutype).flatten()
        channel_2mu2e = (singleMuljEvents&muljInLeading2Events).astype(int)*1
        
        doubleMuljEvents = dileptonjets.ismutype.sum()==2
        muljIsLeading2Events = (lj0.ismutype & lj1.ismutype).flatten()
        channel_4mu = (doubleMuljEvents&muljIsLeading2Events).astype(int)*2
        
        channel_ = channel_2mu2e + channel_4mu
        ###########
        
        output['run'] += processor.column_accumulator(run[(channel_==1)&wgt.astype(bool)])
        output['lumi'] += processor.column_accumulator(lumi[(channel_==1)&wgt.astype(bool)])
        output['event'] += processor.column_accumulator(event[(channel_==1)&wgt.astype(bool)])
        
        
        return output
    
    def postprocess(self, accumulator):
        return accumulator

In [5]:
outputs={}
for k in datasets:
    outputs[k] = processor.run_uproot_job({k: datasets[k]},
                                  treename='ffNtuplizer/ffNtuple',
                                  processor_instance=LeptonJetProcessor(),
                                  executor=processor.futures_executor,
                                  executor_args=dict(workers=12, flatten=True),
                                  chunksize=500000,
                                 )

Processing: 100%|██████████| 1255/1255 [00:39<00:00, 31.98items/s]


In [6]:
for k, output in outputs.items():
    runs, lumis, events = output['run'].value.astype(int), output['lumi'].value.astype(int), output['event'].value.astype(int)
    print(f'\n{k}')
    if runs.size==0: continue
    for r,l,e in np.nditer([runs, lumis, events]):
        print('{}:{}:{}'.format(r, l, e))


A

B

C
319756:601:977673860
319853:6:9168989
319678:58:45364090
319639:982:1413928370
319625:143:213025512
319639:261:391731292
319639:569:882209127
319639:79:86908330
319756:346:537459401
319941:117:144973429
319659:1:720753
319639:841:1246096773
319756:457:727605939
319910:632:1119786183
319849:219:362902045
319849:222:367155844
319849:254:423263267
319840:148:205548107
319993:124:206127085
319991:149:187377297
319912:5:8541510
320002:87:73435218
319991:442:659338315
319993:690:1082058716
319991:614:946384293
319991:136:165222904
320023:261:400546231
320038:531:816400166
320026:103:153982112
320010:106:155950748
319993:949:1468184959
320006:153:258209296
320002:99:84522844
320038:135:166617945
320038:187:252029702
320038:230:318751472
320023:72:93179415
320059:76:123897404
320059:92:149253888
320040:200:319484271
320065:60:101372864
320038:620:969455946
320064:120:194347303
319656:271:344332769
319528:52:79960855

D
320917:1896:2982713313
320917:814:1327690140
320917:600:990617503
